<a href="https://colab.research.google.com/github/Thrishankkuntimaddi/Traffic-Light-Simulation-and-Adjustment/blob/main/Traffic%20Light%20Simulation%20%26%20Adjustment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
import time

def simulate_traffic_data():
    traffic_data = {
        'North': random.randint(10, 50),
        'South': random.randint(10, 50),
        'East': random.randint(10, 50),
        'West': random.randint(10, 50)
    }
    return traffic_data


In [3]:
def adjust_traffic_lights(traffic_data):
    total_traffic = sum(traffic_data.values())
    timings = {}

    for direction, vehicles in traffic_data.items():
        timings[direction] = (vehicles / total_traffic) * 100

    return timings


In [13]:
def monitor_traffic_lights():
    count = 0
    while True:

        print("Monitoring traffic...")
        traffic_data = simulate_traffic_data()
        print(f"Traffic Data: {traffic_data}")

        timings = adjust_traffic_lights(traffic_data)
        print(f"Adjusted Traffic Light Timings: {timings}")

        for direction, timing in timings.items():
            print(f"Green light for {direction} direction for {timing:.2f}% of the time.")

        count += 1

        if count == 4:
            break

In [14]:
monitor_traffic_lights()

Monitoring traffic...
Traffic Data: {'North': 35, 'South': 39, 'East': 49, 'West': 13}
Adjusted Traffic Light Timings: {'North': 25.735294117647058, 'South': 28.676470588235293, 'East': 36.029411764705884, 'West': 9.558823529411764}
Green light for North direction for 25.74% of the time.
Green light for South direction for 28.68% of the time.
Green light for East direction for 36.03% of the time.
Green light for West direction for 9.56% of the time.
Monitoring traffic...
Traffic Data: {'North': 39, 'South': 28, 'East': 12, 'West': 37}
Adjusted Traffic Light Timings: {'North': 33.62068965517241, 'South': 24.137931034482758, 'East': 10.344827586206897, 'West': 31.896551724137932}
Green light for North direction for 33.62% of the time.
Green light for South direction for 24.14% of the time.
Green light for East direction for 10.34% of the time.
Green light for West direction for 31.90% of the time.
Monitoring traffic...
Traffic Data: {'North': 32, 'South': 27, 'East': 12, 'West': 46}
Adju

In [ ]:
import pandas as pd

file_path = '/content/TrafficTwoMonth.csv'
traffic_data = pd.read_csv(file_path)

traffic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5952 entries, 0 to 5951
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Time               5952 non-null   object
 1   Date               5952 non-null   int64 
 2   Day of the week    5952 non-null   object
 3   CarCount           5952 non-null   int64 
 4   BikeCount          5952 non-null   int64 
 5   BusCount           5952 non-null   int64 
 6   TruckCount         5952 non-null   int64 
 7   Total              5952 non-null   int64 
 8   Traffic Situation  5952 non-null   object
dtypes: int64(6), object(3)
memory usage: 418.6+ KB


In [ ]:
traffic_data.head()

,Time,Date,Day of the week,CarCount,BikeCount,BusCount,TruckCount,Total,Traffic Situation
0,12:00:00 AM,10,Tuesday,13,2,2,24,41,normal
1,12:15:00 AM,10,Tuesday,14,1,1,36,52,normal
2,12:30:00 AM,10,Tuesday,10,2,2,32,46,normal
3,12:45:00 AM,10,Tuesday,10,2,2,36,50,normal
4,1:00:00 AM,10,Tuesday,11,2,1,34,48,normal


In [ ]:
traffic_data['Time'] = pd.to_datetime(traffic_data['Time'], format='%I:%M:%S %p').dt.time

traffic_data['Hour'] = pd.to_datetime(traffic_data['Time'].astype(str)).dt.hour
average_traffic_by_hour = traffic_data.groupby('Hour')['Total'].mean()

average_traffic_by_day = traffic_data.groupby('Day of the week')['Total'].mean()

traffic_situation_distribution = traffic_data['Traffic Situation'].value_counts()

average_traffic_by_hour, average_traffic_by_day, traffic_situation_distribution

<ipython-input-17-c794c620db69>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  traffic_data['Hour'] = pd.to_datetime(traffic_data['Time'].astype(str)).dt.hour


(Hour
 0      42.762097
 1      42.419355
 2      43.290323
 3      44.524194
 4      98.036290
 5     113.012097
 6     163.798387
 7     163.415323
 8     164.899194
 9     120.975806
 10    113.919355
 11    102.266129
 12    102.907258
 13    118.431452
 14    120.987903
 15    118.790323
 16    177.314516
 17    175.939516
 18    158.213710
 19    119.806452
 20    118.766129
 21    110.540323
 22     42.588710
 23     42.270161
 Name: Total, dtype: float64,
 Day of the week
 Friday       106.283854
 Monday       108.822917
 Saturday     110.080729
 Sunday       109.636719
 Thursday     109.452083
 Tuesday      109.252083
 Wednesday    110.237500
 Name: Total, dtype: float64,
 Traffic Situation
 normal    3610
 heavy     1137
 low        834
 high       371
 Name: count, dtype: int64)

In [ ]:
def adjust_traffic_light_timing(hour, traffic_situation, emergency_vehicle=False):
    base_time = 60

    if 6 <= hour <= 9 or 16 <= hour <= 18:
        green_light_time = base_time * 1.5
    elif 22 <= hour <= 3:
        green_light_time = base_time * 0.5
    else:
        green_light_time = base_time

    if traffic_situation == "heavy":
        green_light_time *= 1.5
    elif traffic_situation == "low":
        green_light_time *= 0.75
    elif traffic_situation == "high":
        green_light_time *= 1.25

    if emergency_vehicle:
        green_light_time = base_time * 2

    return green_light_time

current_hour = 8
current_traffic_situation = 'heavy'
emergency_vehicle_detected = True

adjusted_timing = adjust_traffic_light_timing(current_hour, current_traffic_situation, emergency_vehicle_detected)
adjusted_timing

120